# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
# prompt: library opencv

#!pip install opencv-python


In [ ]:
#!pip install ultralytics
#!pip install opencv-p

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [24]:
from ultralytics import YOLO,solutions

model = YOLO('/content/YOLO_Model.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 69.5MB/s]


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
import cv2
cap = cv2.VideoCapture('/content/YOLO_Video.mp4')

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

print(f"Video width: {width}, height: {height}, FPS: {fps}")

Video width: 1920, height: 1080, FPS: 29.97002997002997


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
output_path = "YOLO_Video_Output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [ ]:
# Define ROIs (example coordinates, adjust as necessary)
roi_top = (0, 0, width, int(height * 0.2))  # Top ROI
roi_bottom = (0, int(height * 0.8), width, height)  # Bottom ROI
roi_left = (0, 0, int(width * 0.2), height)  # Left ROI
roi_right = (int(width * 0.8), 0, width, height)  # Right ROI

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:

# Initialize counters and tracking dictionary
count_top, count_bottom, count_left, count_right = 0, 0, 0, 0
vehicle_tracker = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model
    results = model(frame)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
            vehicle_id = box.id  # Assuming box.id provides a unique ID for each detected object

            # If the vehicle is new, add it to the tracker
            if vehicle_id not in vehicle_tracker:
                vehicle_tracker[vehicle_id] = {"counted": False}

            # Check if vehicle is crossing any ROI
            if not vehicle_tracker[vehicle_id]["counted"]:
                if roi_top[1] < cy < roi_top[3] and roi_top[0] < cx < roi_top[2]:
                    count_top += 1
                    vehicle_tracker[vehicle_id]["counted"] = True
                elif roi_bottom[1] < cy < roi_bottom[3] and roi_bottom[0] < cx < roi_bottom[2]:
                    count_bottom += 1
                    vehicle_tracker[vehicle_id]["counted"] = True
                elif roi_left[1] < cy < roi_left[3] and roi_left[0] < cx < roi_left[2]:
                    count_left += 1
                    vehicle_tracker[vehicle_id]["counted"] = True
                elif roi_right[1] < cy < roi_right[3] and roi_right[0] < cx < roi_right[2]:
                    count_right += 1
                    vehicle_tracker[vehicle_id]["counted"] = True

            # Draw bounding box and center point
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    # Display the ROI rectangles
    cv2.rectangle(frame, (roi_top[0], roi_top[1]), (roi_top[2], roi_top[3]), (255, 0, 0), 2)
    cv2.rectangle(frame, (roi_bottom[0], roi_bottom[1]), (roi_bottom[2], roi_bottom[3]), (255, 0, 0), 2)
    cv2.rectangle(frame, (roi_left[0], roi_left[1]), (roi_left[2], roi_left[3]), (255, 0, 0), 2)
    cv2.rectangle(frame, (roi_right[0], roi_right[1]), (roi_right[2], roi_right[3]), (255, 0, 0), 2)

    # Display vehicle counts
    cv2.putText(frame, f'Top: {count_top}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f'Bottom: {count_bottom}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f'Left: {count_left}', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f'Right: {count_right}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Write the frame to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()

print("Processing complete. Output video saved.")



0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 2738.2ms
Speed: 3.8ms preprocess, 2738.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 3821.6ms
Speed: 3.8ms preprocess, 3821.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 2857.5ms
Speed: 4.2ms preprocess, 2857.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2699.4ms
Speed: 3.9ms preprocess, 2699.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 2691.6ms
Speed: 3.7ms preprocess, 2691.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 3059.9ms
Speed: 3.7ms preprocess, 3059.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 3648.4ms
Speed: 4.1ms p

## Save and Submit Your Work

Submit both the notebook and the output video